<a href="https://colab.research.google.com/github/Tristan-Brown1096/DS18_Unit_2_Build_Week_Project/blob/master/unit_2_build_week_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data wrangling

In [ ]:
import pandas as pd
import numpy as np
!pip install category_encoders==2.*
!pip install pandas-profiling==2.*

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Tristan-Brown1096/DS18_Unit_2_Build_Week_Project/master/data/cbb.csv')

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 0.1,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,0,0.0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,1,1.0,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,2,2.0,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,3,3.0,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,4,4.0,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [3]:
df.TEAM = [team.replace(' ', '') for team in df.TEAM]

In [4]:
train = df[df['YEAR'] <= 2017]
val = df[df['YEAR'] == 2018]
test = df[df['YEAR'] == 2019]

In [5]:
train['YEAR'] = train['YEAR'].astype(str)
val['YEAR'] = val['YEAR'].astype(str)
test['YEAR'] = test['YEAR'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [6]:
train['ID'] = train.TEAM + train.YEAR
val['ID'] = val.TEAM + val.YEAR
test['ID'] = test.TEAM + test.YEAR

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [7]:
train = train.set_index('ID')
val = val.set_index('ID')
test = test.set_index('ID')

In [ ]:
train.head()

,Unnamed: 0,Unnamed: 0.1,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
NorthCarolina2016,0,0.0,NorthCarolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
Wisconsin2015,1,1.0,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
Gonzaga2017,4,4.0,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017
Duke2015,5,5.0,Duke,ACC,39,35,125.2,90.6,0.9764,56.6,46.5,16.3,18.6,35.8,30.2,39.8,23.9,55.9,46.3,38.7,31.4,66.4,10.7,Champions,1.0,2015
NorthCarolina2017,7,7.0,NorthCarolina,ACC,39,33,121.0,91.5,0.9615,51.7,48.1,16.2,18.6,41.3,25.0,34.3,31.6,51.0,46.3,35.5,33.9,72.8,8.4,Champions,1.0,2017


#Split data

In [8]:
target = 'POSTSEASON'

In [9]:
X_train = train.drop([target, 'TEAM', 'YEAR', 'SEED', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_train = train[target]

X_val = val.drop([target, 'TEAM', 'YEAR', 'SEED', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_val = val[target]

X_test = test.drop([target, 'TEAM', 'YEAR', 'SEED', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_test = test[target]

#Baseline

In [10]:
baseline = y_train.value_counts(normalize=True).max()
print('Baseline Accuracy:', baseline)

Baseline Accuracy: 0.8058844635809114


#Build model_1

In [17]:
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [18]:
model_1 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_depth=4)
)

model_1.fit(X_train, y_train)
print('Training Accuracy:', model_1.score(X_train, y_train))
print('Validation Accuracy:', model_1.score(X_val, y_val))

Training Accuracy: 0.8697524219590959
Validation Accuracy: 0.8603988603988604


In [19]:
from sklearn.metrics import accuracy_score
print('Test Accuracy', accuracy_score(y_test, model_1.predict(X_test)))

Test Accuracy 0.8498583569405099


#Tuning

In [22]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [147]:
model_2 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier()
)

params = {'randomforestclassifier__n_estimators': range(100, 300, 50),
          'randomforestclassifier__max_depth': range(10, 20, 2),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

gs = GridSearchCV(model_2, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [148]:
gs.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

In [ ]:
gs.best_params_

{'randomforestclassifier__max_depth': 10,
 'randomforestclassifier__max_features': 'log2',
 'randomforestclassifier__n_estimators': 150}

#Build model_2

In [ ]:
model_2 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(max_depth=10, max_features='log2', n_estimators=150)
)
model_2.fit(X_train, y_train)
print('Training accuracy:', model_2.score(X_train, y_train))
print('Validation accuracy:', model_2.score(X_val, y_val))
print('Test Accuracy', accuracy_score(y_test, model_2.predict(X_test)))

Training accuracy: 0.9752421959095802
Validation accuracy: 0.8490028490028491
Test Accuracy 0.8498583569405099


#More tuning

In [ ]:
params = {'randomforestclassifier__n_estimators': range(50, 200, 10),
          'randomforestclassifier__max_depth': range(2, 10, 2),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

gs = GridSearchCV(model_2, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
gs.fit(X_train, y_train)
gs.best_params_

Fitting 5 folds for each of 180 candidates, totalling 900 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   39.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  4.6min finished


{'randomforestclassifier__max_depth': 4,
 'randomforestclassifier__max_features': None,
 'randomforestclassifier__n_estimators': 80}

#Build model_3

In [ ]:
model_3 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=80, max_depth=4, max_features=None)
)

model_3.fit(X_train, y_train)
print('Training accuracy:', model_3.score(X_train, y_train))
print('Validation accuracy:', model_3.score(X_val, y_val))
print('Test Accuracy', accuracy_score(y_test, model_3.predict(X_test)))

Training accuracy: 0.8776462145676355
Validation accuracy: 0.8717948717948718
Test Accuracy 0.8555240793201133


#Tuning again

In [ ]:
params = {'randomforestclassifier__n_estimators': range(70, 90),
          'randomforestclassifier__max_depth': range(1, 10),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

model_4 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier()
)

gs = GridSearchCV(model_4, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
gs.fit(X_train, y_train)
gs.best_params_

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 2700 out of 2700 | elapsed:  9.5min finished


{'randomforestclassifier__max_depth': 4,
 'randomforestclassifier__max_features': 'log2',
 'randomforestclassifier__n_estimators': 79}

#Build model_4

In [ ]:
model_4 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=79, max_depth=4, max_features='log2')
)

model_4.fit(X_train, y_train)
print('Training accuracy:', model_4.score(X_train, y_train))
print('Validation accuracy:', model_4.score(X_val, y_val))
print('Test Accuracy', accuracy_score(y_test, model_4.predict(X_test)))

Training accuracy: 0.8701112307140294
Validation accuracy: 0.8575498575498576
Test Accuracy 0.8526912181303116


#Build model_5

In [ ]:
model_5 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_depth=5, max_features=None)
)

model_5.fit(X_train, y_train)
print('Training accuracy:', model_5.score(X_train, y_train))
print('Validation accuracy:', model_5.score(X_val, y_val))
print('Test Accuracy', accuracy_score(y_test, model_5.predict(X_test)))

Training accuracy: 0.8959454610692501
Validation accuracy: 0.8717948717948718
Test Accuracy 0.8583569405099151


#Build model_6

In [ ]:
model_6 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42)
)

model_6.fit(X_train, y_train)
print('Training accuracy:', model_6.score(X_train, y_train))
print('Validation accuracy:', model_6.score(X_val, y_val))
print('Test Accuracy', accuracy_score(y_test, model_6.predict(X_test)))

Training accuracy: 1.0
Validation accuracy: 0.8490028490028491
Test Accuracy 0.8526912181303116


#Accuracy scores

In [ ]:
print('Model_1 Test Accuracy', accuracy_score(y_test, model_1.predict(X_test)))
print('Model_2 Test Accuracy', accuracy_score(y_test, model_2.predict(X_test)))
print('Model_3 Test Accuracy', accuracy_score(y_test, model_3.predict(X_test)))
print('Model_4 Test Accuracy', accuracy_score(y_test, model_4.predict(X_test)))
print('Model_5 Test Accuracy', accuracy_score(y_test, model_5.predict(X_test)))
print('Model_6 Test Accuracy', accuracy_score(y_test, model_6.predict(X_test)))

Model_1 Test Accuracy 0.8526912181303116
Model_2 Test Accuracy 0.8498583569405099
Model_3 Test Accuracy 0.8555240793201133
Model_4 Test Accuracy 0.8526912181303116
Model_5 Test Accuracy 0.8583569405099151
Model_6 Test Accuracy 0.8526912181303116


#Look at predictions

In [ ]:
predictions_2019 = pd.DataFrame(model_3.predict(X_test), index=X_test.index, columns=['prediction'])
predictions_2019 = pd.concat([predictions_2019, y_test], axis=1)
predictions_2019.head()

,prediction,POSTSEASON
ID,,
TexasTech2019,E8,2ND
Virginia2019,E8,Champions
Duke2019,E8,E8
Purdue2019,R32,E8
Kentucky2019,E8,E8


#Export predictions

In [ ]:
predictions_2019.to_csv('predictions.csv')

#Other metrics

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

In [ ]:
print('Model_1 F1 Score:', f1_score(y_test, model_1.predict(X_test), average='micro'))
print('Model_2 F1 Score:', f1_score(y_test, model_2.predict(X_test), average='micro'))
print('Model_3 F1 Score:', f1_score(y_test, model_3.predict(X_test), average='micro'))
print('Model_4 F1 Score:', f1_score(y_test, model_4.predict(X_test), average='micro'))
print('Model_5 F1 Score:', f1_score(y_test, model_5.predict(X_test), average='micro'))
print('Model_6 F1 Score:', f1_score(y_test, model_6.predict(X_test), average='micro'))

Model_1 F1 Score: 0.8526912181303116
Model_2 F1 Score: 0.8498583569405099
Model_3 F1 Score: 0.8555240793201133
Model_4 F1 Score: 0.8526912181303116
Model_5 F1 Score: 0.8583569405099151
Model_6 F1 Score: 0.8526912181303116


#Only tournament teams

In [11]:
df_1 = df[df['POSTSEASON'] != 'no tourney']
df_1.head()

,Unnamed: 0,Unnamed: 0.1,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,0,0.0,NorthCarolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,1,1.0,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,2,2.0,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,3,3.0,TexasTech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,4,4.0,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [12]:
train_1 = df_1[df_1['YEAR'] <= 2017]
val_1 = df_1[df_1['YEAR'] == 2018]
test_1 = df_1[df_1['YEAR'] == 2019]

In [13]:
train_1['YEAR'] = train_1['YEAR'].astype(str)
val_1['YEAR'] = val_1['YEAR'].astype(str)
test_1['YEAR'] = test_1['YEAR'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [14]:
train_1['ID'] = train_1['TEAM'] + train_1['YEAR']
val_1['ID'] = val_1.TEAM + val_1.YEAR
test_1['ID'] = test_1.TEAM + test_1.YEAR

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [15]:
train_1 = train_1.set_index('ID')
val_1 = val_1.set_index('ID')
test_1 = test_1.set_index('ID')

train_1.head()

,Unnamed: 0,Unnamed: 0.1,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
NorthCarolina2016,0,0.0,NorthCarolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
Wisconsin2015,1,1.0,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
Gonzaga2017,4,4.0,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017
Duke2015,5,5.0,Duke,ACC,39,35,125.2,90.6,0.9764,56.6,46.5,16.3,18.6,35.8,30.2,39.8,23.9,55.9,46.3,38.7,31.4,66.4,10.7,Champions,1.0,2015
NorthCarolina2017,7,7.0,NorthCarolina,ACC,39,33,121.0,91.5,0.9615,51.7,48.1,16.2,18.6,41.3,25.0,34.3,31.6,51.0,46.3,35.5,33.9,72.8,8.4,Champions,1.0,2017


In [16]:
X_train_1 = train_1.drop([target, 'TEAM', 'YEAR', 'SEED', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_train_1 = train_1[target]

X_val_1 = val_1.drop([target, 'TEAM', 'YEAR', 'SEED', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_val_1 = val_1[target]

X_test_1 = test_1.drop([target, 'TEAM', 'YEAR', 'SEED', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_test_1 = test_1[target]

#New Baseline

In [51]:
baseline_1 = y_train_1.value_counts(normalize=True).max()
print('Baseline Accuracy:', baseline_1)

Baseline Accuracy: 0.4731977818853974


#Build model_7

In [ ]:
model_7 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_depth=5, max_features=None)
)

model_7.fit(X_train_1, y_train_1)
print('Training accuracy:', model_7.score(X_train_1, y_train_1))
print('Validation accuracy:', model_7.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_7.predict(X_test_1)))

Training accuracy: 0.7966728280961183
Validation accuracy: 0.6617647058823529
Test Accuracy 0.5882352941176471


#Build model_8

In [ ]:
model_8 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=80, max_depth=4, max_features=None)
)

model_8.fit(X_train_1, y_train_1)
print('Training accuracy:', model_8.score(X_train_1, y_train_1))
print('Validation accuracy:', model_8.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_8.predict(X_test_1)))

Training accuracy: 0.7060998151571165
Validation accuracy: 0.6617647058823529
Test Accuracy 0.5441176470588235


#Tune model

In [ ]:
model_9 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=42, n_estimators=80, max_depth=4, max_features=None)
)

params = {'randomforestclassifier__n_estimators': range(100, 300, 50),
          'randomforestclassifier__max_depth': range(10, 20, 2),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

gs = GridSearchCV(model_9, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
gs.fit(X_train_1, y_train_1)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ordinalencoder',
                                        OrdinalEncoder(cols=None,
                                                       drop_invariant=False,
                                                       handle_missing='value',
                                                       handle_unknown='value',
                                                       mapping=None,
                                                       return_df=True,
                                                       verbose=0)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                               

In [ ]:
gs.best_params_

{'randomforestclassifier__max_depth': 10,
 'randomforestclassifier__max_features': 'sqrt',
 'randomforestclassifier__n_estimators': 100}

#Build model_9

In [ ]:
model_9 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=42, n_estimators=100, max_depth=4, max_features='sqrt')
)
model_9.fit(X_train_1, y_train_1)
print('Training accuracy:', model_9.score(X_train_1, y_train_1))
print('Validation accuracy:', model_9.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_9.predict(X_test_1)))

Training accuracy: 0.6635859519408502
Validation accuracy: 0.6029411764705882
Test Accuracy 0.5294117647058824


#More tuning

In [ ]:
model_10 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier()
)

params = {'randomforestclassifier__n_estimators': range(0, 100, 10),
          'randomforestclassifier__max_depth': range(0, 11),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

gs = GridSearchCV(model_10, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
gs.fit(X_train_1, y_train_1)
gs.best_params_

Fitting 5 folds for each of 330 candidates, totalling 1650 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 650 tasks      | elapsed:   46.7s
[Parallel(n_jobs=-1)]: Done 1138 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1650 out of 1650 | elapsed:  2.8min finished


{'randomforestclassifier__max_depth': 3,
 'randomforestclassifier__max_features': None,
 'randomforestclassifier__n_estimators': 50}

#Build model_10

In [ ]:
model_10 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(n_estimators=50, max_features=None, max_depth=3, random_state=42)
)
model_10.fit(X_train_1, y_train_1)
print('Training accuracy:', model_10.score(X_train_1, y_train_1))
print('Validation accuracy:', model_10.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_10.predict(X_test_1)))

Training accuracy: 0.6543438077634011
Validation accuracy: 0.6470588235294118
Test Accuracy 0.5735294117647058


#Further tuning

In [ ]:
model_11 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier()
)

params = {'randomforestclassifier__n_estimators': range(25, 75, 5),
          'randomforestclassifier__max_depth': range(0, 11),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}
gs = GridSearchCV(model_11, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
gs.fit(X_train_1, y_train_1)
gs.best_params_

Fitting 5 folds for each of 330 candidates, totalling 1650 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 656 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-1)]: Done 906 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1256 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1650 out of 1650 | elapsed:  2.9min finished


{'randomforestclassifier__max_depth': 3,
 'randomforestclassifier__max_features': None,
 'randomforestclassifier__n_estimators': 40}

#Build model_11

In [ ]:
model_11 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(n_estimators=40, max_features=None, max_depth=3, random_state=42)
)
model_11.fit(X_train_1, y_train_1)
print('Training accuracy:', model_11.score(X_train_1, y_train_1))
print('Validation accuracy:', model_11.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_11.predict(X_test_1)))

Training accuracy: 0.6487985212569316
Validation accuracy: 0.6470588235294118
Test Accuracy 0.5588235294117647


#Build model_12


In [ ]:
model_12 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(n_estimators=300, max_features='sqrt')
)
model_12.fit(X_train_1, y_train_1)
print('Training accuracy:', model_12.score(X_train_1, y_train_1))
print('Validation accuracy:', model_12.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_12.predict(X_test_1)))

Training accuracy: 1.0
Validation accuracy: 0.5588235294117647
Test Accuracy 0.5441176470588235


#Test accuracies

In [ ]:
print('Model_7 Test Accuracy', accuracy_score(y_test_1, model_7.predict(X_test_1)))
print('Model_8 Test Accuracy', accuracy_score(y_test_1, model_8.predict(X_test_1)))
print('Model_9 Test Accuracy', accuracy_score(y_test_1, model_9.predict(X_test_1)))
print('Model_10 Test Accuracy', accuracy_score(y_test_1, model_10.predict(X_test_1)))
print('Model_11 Test Accuracy', accuracy_score(y_test_1, model_11.predict(X_test_1)))
print('Model_12 Test Accuracy', accuracy_score(y_test_1, model_12.predict(X_test_1)))

Model_7 Test Accuracy 0.5882352941176471
Model_8 Test Accuracy 0.5441176470588235
Model_9 Test Accuracy 0.5294117647058824
Model_10 Test Accuracy 0.5735294117647058
Model_11 Test Accuracy 0.5588235294117647
Model_12 Test Accuracy 0.5441176470588235


#Look at new predictions

In [ ]:
predictions_2019_v2 = pd.DataFrame(model_7.predict(X_test_1), index=X_test_1.index, columns=['prediction'])
predictions_2019_v2 = pd.concat([predictions_2019_v2, y_test_1], axis=1)
predictions_2019_v2.head()

,prediction,POSTSEASON
ID,,
TexasTech2019,E8,2ND
Virginia2019,E8,Champions
Duke2019,E8,E8
Purdue2019,R32,E8
Kentucky2019,E8,E8


In [ ]:
predictions_2019_v2.to_csv('predictions_v2.csv')

In [ ]:
predictions_2019_v2['prediction'].value_counts()

R64    41
R32    16
E8      8
2ND     1
S16     1
F4      1
Name: prediction, dtype: int64

#Build model_13

In [ ]:
model_13 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_depth=5, max_features=None, criterion='entropy')
)

model_13.fit(X_train_1, y_train_1)
print('Training accuracy:', model_13.score(X_train_1, y_train_1))
print('Validation accuracy:', model_13.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_13.predict(X_test_1)))

Training accuracy: 0.7929759704251387
Validation accuracy: 0.5882352941176471
Test Accuracy 0.6029411764705882


In [ ]:
predictions_2019_v3 = pd.DataFrame(model_13.predict(X_test_1), index=X_test_1.index, columns=['prediction'])
predictions_2019_v3 = pd.concat([predictions_2019_v3, y_test_1], axis=1)
predictions_2019_v3.head()

,prediction,POSTSEASON
ID,,
TexasTech2019,E8,2ND
Virginia2019,E8,Champions
Duke2019,E8,E8
Purdue2019,R32,E8
Kentucky2019,E8,E8


In [ ]:
predictions_2019_v3['prediction'].value_counts()

R64    41
R32    16
E8      7
S16     2
2ND     1
F4      1
Name: prediction, dtype: int64

#Build model_14

In [ ]:
model_14 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=42, criterion='entropy')
)

model_14.fit(X_train_1, y_train_1)
print('Training accuracy:', model_14.score(X_train_1, y_train_1))
print('Validation accuracy:', model_14.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_14.predict(X_test_1)))

Training accuracy: 1.0
Validation accuracy: 0.5441176470588235
Test Accuracy 0.5588235294117647


#Tune model

In [ ]:
model_14 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=42, criterion='entropy')
)
params = {'randomforestclassifier__n_estimators': range(0, 300, 10),
          'randomforestclassifier__max_depth': range(0, 20, 2),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

gs = GridSearchCV(model_14, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
gs.fit(X_train_1, y_train_1)
gs.best_params_

[Parallel(n_jobs=-1)]: Done 1172 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 1622 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 2172 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 2822 tasks      | elapsed: 20.5min
[Parallel(n_jobs=-1)]: Done 3572 tasks      | elapsed: 28.4min
[Parallel(n_jobs=-1)]: Done 4422 tasks      | elapsed: 36.7min
[Parallel(n_jobs=-1)]: Done 4500 out of 4500 | elapsed: 38.8min finished


{'randomforestclassifier__max_depth': 4,
 'randomforestclassifier__max_features': None,
 'randomforestclassifier__n_estimators': 100}

#Build model_15

In [ ]:
model_15 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=42, criterion='entropy', max_features=None, max_depth=4)
)
model_15.fit(X_train_1, y_train_1)
print('Training accuracy:', model_15.score(X_train_1, y_train_1))
print('Validation accuracy:', model_15.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_15.predict(X_test_1)))

Training accuracy: 0.6876155268022182
Validation accuracy: 0.6176470588235294
Test Accuracy 0.5735294117647058


#Build model_16

In [20]:
from xgboost import XGBClassifier

In [21]:
model_16 = make_pipeline(
    OrdinalEncoder(),
    XGBClassifier()
)
model_16.fit(X_train_1, y_train_1)
print('Training accuracy:', model_16.score(X_train_1, y_train_1))
print('Validation accuracy:', model_16.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_16.predict(X_test_1)))

Training accuracy: 0.9482439926062847
Validation accuracy: 0.6029411764705882
Test Accuracy 0.5294117647058824


#Build model_17

In [50]:
model_17 = make_pipeline(
    OrdinalEncoder(),
    XGBClassifier(max_depth=5, learning_rate=0.001, n_estimators=500, n_jobs=-1,
                  objective='multi:sotmax', eval_metric='merror', num_class=8,
                  critereon = 'entropy')
)
model_17.fit(X_train_1, y_train_1)
print('Training accuracy:', model_17.score(X_train_1, y_train_1))
print('Validation accuracy:', model_17.score(X_val_1, y_val_1))
print('Test Accuracy', accuracy_score(y_test_1, model_17.predict(X_test_1)))

Training accuracy: 0.7948243992606284
Validation accuracy: 0.5588235294117647
Test Accuracy 0.6029411764705882


#Tourney Teams with seeds

In [59]:
X_train_2 = train_1.drop([target, 'TEAM', 'YEAR', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_train_2 = train_1[target]

X_val_2 = val_1.drop([target, 'TEAM', 'YEAR', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_val_2 = val_1[target]

X_test_2 = test_1.drop([target, 'TEAM', 'YEAR', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
y_test_2 = test_1[target]

In [62]:
model_18 = make_pipeline(
    OrdinalEncoder(),
    XGBClassifier(max_depth=5, learning_rate=0.001, n_estimators=500, n_jobs=-1,
                  objective='multi:sotmax', eval_metric='merror', num_class=8,
                  critereon = 'entropy')
)
model_18.fit(X_train_2, y_train_2)
print('Training accuracy:', model_18.score(X_train_2, y_train_2))
print('Validation accuracy:', model_18.score(X_val_2, y_val_2))
print('Test Accuracy', accuracy_score(y_test_2, model_18.predict(X_test_2)))

Training accuracy: 0.8059149722735675
Validation accuracy: 0.6029411764705882
Test Accuracy 0.6029411764705882


In [68]:
y_train_2.value_counts()

R64          256
R32          128
S16           64
E8            32
R68           29
F4            16
2ND            8
Champions      8
Name: POSTSEASON, dtype: int64

In [175]:
pred = pd.DataFrame(model_18.predict_proba(X_test_2), index=X_test_2.index)
pred.head()

,0,1,2,3,4,5,6,7
ID,,,,,,,,
TexasTech2019,0.130625,0.097265,0.182459,0.193569,0.098776,0.098098,0.097406,0.101802
Virginia2019,0.184175,0.105371,0.162716,0.114430,0.111226,0.106273,0.105524,0.110286
Duke2019,0.140813,0.109086,0.187967,0.117912,0.110781,0.110020,0.109245,0.114175
Purdue2019,0.090350,0.089889,0.138280,0.089932,0.196662,0.090658,0.090019,0.214209
Kentucky2019,0.095768,0.095279,0.266251,0.138119,0.096759,0.096094,0.095417,0.116313


In [119]:
pred.sort_values(by=5, ascending=False).head()

,0,1,2,3,4,5,6,7
ID,,,,,,,,
Bradley2019,0.096153,0.095662,0.095731,0.095709,0.097090,0.327126,0.096530,0.095998
Northeastern2019,0.094574,0.094091,0.094159,0.094137,0.111918,0.321754,0.094945,0.094422
GeorgiaSt.2019,0.094553,0.094070,0.094138,0.094116,0.112118,0.321681,0.094923,0.094401
Montana2019,0.095909,0.095420,0.095488,0.095466,0.107691,0.317984,0.096285,0.095755
SetonHall2019,0.093366,0.092889,0.092956,0.092934,0.123971,0.317643,0.093024,0.093216


In [183]:
p18 = pd.DataFrame(model_18.predict(X_test_2), index=X_test_2.index, columns=['Prediction'])
p18 = pd.concat([p18, y_test_2], axis=1)
p18[p18['Prediction'] == 'E8']

,Prediction,POSTSEASON
ID,,
Duke2019,E8,E8
Kentucky2019,E8,E8
FloridaSt.2019,E8,S16
Houston2019,E8,S16


In [133]:
pred.loc['ArizonaSt.2019']

2ND          0.086605
Champions    0.086163
E8           0.086225
F4           0.086204
R32          0.151821
R64          0.294641
R68          0.121876
S16          0.086465
Name: ArizonaSt.2019, dtype: float32

#Build model_19

In [136]:
from sklearn.ensemble import GradientBoostingClassifier

In [138]:
model_19 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    GradientBoostingClassifier()
)
model_19.fit(X_train_2, y_train_2)
print('Training accuracy:', model_19.score(X_train_2, y_train_2))
print('Validation accuracy:', model_19.score(X_val_2, y_val_2))
print('Test Accuracy', accuracy_score(y_test_2, model_19.predict(X_test_2)))

Training accuracy: 1.0
Validation accuracy: 0.5588235294117647
Test Accuracy 0.5735294117647058


#Tune from model_19

In [162]:
params = {
    'gradientboostingclassifier__learning_rate' : [0.001, 0.003, 0.005, 0/007, 0.009],
    'gradientboostingclassifier__max_depth' : range(1, 10, 2),
    'gradientboostingclassifier__n_estimators' : range(100, 1000, 200)
}
gs = GridSearchCV(model_20, params, scoring='accuracy', n_jobs=-1, verbose=1)

In [156]:
gs.fit(X_train_2, y_train_2)

Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 78.5min
[Parallel(n_jobs=-1)]: Done 625 out of 625 | elapsed: 111.3min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ordinalencoder',
                                        OrdinalEncoder(cols=['CONF'],
                                                       drop_invariant=False,
                                                       handle_missing='value',
                                                       handle_unknown='value',
                                                       mapping=[{'col': 'CONF',
                                                                 'data_type': dtype('O'),
                                                                 'mapping': ACC      1
B10      2
WCC      3
BE       4
B12      5
P12      6
SEC      7
A10      8
Amer     9
BW      10
CUSA    11
Ivy     12
MVC     13
MWC     14
SB      15
Slnd    16
AE      17
ASun    18
BSky    19
BSth    20
CAA     21
Horz    22
M...
                                                            

In [157]:
gs.best_params_

{'gradientboostingclassifier__learning_rate': 0.003,
 'gradientboostingclassifier__max_depth': 1,
 'gradientboostingclassifier__n_estimators': 900}

#Build model_20

In [160]:
model_20 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    GradientBoostingClassifier(learning_rate=0.003, max_depth=1, n_estimators=900)
)
model_20.fit(X_train_2, y_train_2)
print('Training accuracy:', model_20.score(X_train_2, y_train_2))
print('Validation accuracy:', model_20.score(X_val_2, y_val_2))
print('Test Accuracy', accuracy_score(y_test_2, model_20.predict(X_test_2)))

Training accuracy: 0.6321626617375231
Validation accuracy: 0.6323529411764706
Test Accuracy 0.6029411764705882


#Build model_21

In [161]:
model_21 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    GradientBoostingClassifier(learning_rate=0.003, max_depth=1, n_estimators=500)
)
model_21.fit(X_train_2, y_train_2)
print('Training accuracy:', model_21.score(X_train_2, y_train_2))
print('Validation accuracy:', model_21.score(X_val_2, y_val_2))
print('Test Accuracy', accuracy_score(y_test_2, model_21.predict(X_test_2)))

Training accuracy: 0.6062846580406654
Validation accuracy: 0.5588235294117647
Test Accuracy 0.6029411764705882


#Tune model_21

In [163]:
params = {
    'gradientboostingclassifier__learning_rate' : [0.001, 0.003, 0.005],
    'gradientboostingclassifier__max_depth' : range(1, 5, 2),
    'gradientboostingclassifier__n_estimators' : range(300, 900, 100)
}
gs = GridSearchCV(model_21, params, scoring='accuracy', n_jobs=-1, verbose=1)

In [164]:
gs.fit(X_train_2, y_train_2)
gs.best_params_

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 17.2min finished


{'gradientboostingclassifier__learning_rate': 0.005,
 'gradientboostingclassifier__max_depth': 1,
 'gradientboostingclassifier__n_estimators': 400}

#Build model_22

In [206]:
model_22 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    GradientBoostingClassifier(learning_rate=0.005, max_depth=1, n_estimators=400)
)
model_22.fit(X_train_2, y_train_2)
print('Training accuracy:', model_22.score(X_train_2, y_train_2))
print('Validation accuracy:', model_22.score(X_val_2, y_val_2))
print('Test Accuracy', accuracy_score(y_test_2, model_22.predict(X_test_2)))

Training accuracy: 0.6155268022181146
Validation accuracy: 0.5882352941176471
Test Accuracy 0.6176470588235294


In [207]:
p22 = pd.DataFrame(model_22.predict(X_test_2), index=X_test_2.index, columns=['Prediction'])
p22 = pd.concat([p22, y_test_2], axis=1)
p22[p22['Prediction'] == 'E8']

,Prediction,POSTSEASON
ID,,
TexasTech2019,E8,2ND
Virginia2019,E8,Champions
Duke2019,E8,E8
Kentucky2019,E8,E8
Gonzaga2019,E8,E8
Houston2019,E8,S16
Michigan2019,E8,S16


#Tune model_22

In [166]:
params = {
    'gradientboostingclassifier__learning_rate' : [0.005, 0.006, 0.007],
    'gradientboostingclassifier__max_depth' : range(1, 3),
    'gradientboostingclassifier__n_estimators' : range(400, 600, 20)
}
gs = GridSearchCV(model_22, params, scoring='accuracy', n_jobs=-1, verbose=1)

In [167]:
gs.fit(X_train_2, y_train_2)
gs.best_params_

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 20.7min finished


{'gradientboostingclassifier__learning_rate': 0.005,
 'gradientboostingclassifier__max_depth': 2,
 'gradientboostingclassifier__n_estimators': 460}

#Build model_23

In [170]:
model_23 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    GradientBoostingClassifier(learning_rate=0.005, max_depth=2, n_estimators=460)
)
model_23.fit(X_train_2, y_train_2)
print('Training accuracy:', model_23.score(X_train_2, y_train_2))
print('Validation accuracy:', model_23.score(X_val_2, y_val_2))
print('Test Accuracy', accuracy_score(y_test_2, model_23.predict(X_test_2)))

Training accuracy: 0.7319778188539742
Validation accuracy: 0.6029411764705882
Test Accuracy 0.5735294117647058
